# Chapter06 : 학습 관련 기술들

In [ ]:
import numpy as np
import torch

## 1. 매개변수 갱신

### SGD : Stochastic Gradient Descent

In [ ]:
# SGD : Stochastic Gradient Descent
class SGD:
    def __init__(self, lr=0.01):                    # lr : learning rate를 의미
        self.lr = lr
    
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]     # 각 변수의 주소로 접근해 값을 변경

##### Greedy algorithm : 매 선택에서 최적이라 판단되는 것을 선택하는 알고리즘. 최적 해를 구하는 근사적 방법이지만 단점이 많다.<br>SGD는 매 순간 최대의 효율을 얻는 길을 선택한다.

### Momentum : 운동량
> 물체가 경사면을 따라 운동할때에는 가속도(현재 SGD의 미분값)뿐만 아니라 물체의 속도 또한 존재한다. 이를 구현한 방식이 모멘텀이다.

In [ ]:
class Momentum:
    def __init__(self, lr=0.01, momentum = 0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
    
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
        
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]  

### AdaGrad : 모든 변수에 통일된 학습률을 개별 매개변수에 서로 다른 학습률을 적용하는 방식, 학습률 변화
> 기울기가 컸던 파라미터의 학습률은 낮게, 기울기가 작았던 파라미터의 학습률은 크게

In [ ]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
    
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
        
        for key in params.keys():
            self.h[key] += grads[key] ** 2          # 과거 기울기값 ~ self.h ~ 1/학습량
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-07)



### Adam : Adaptive Moment Estimation
>RMSprop와 Momentum을 기반으로 만들어진 optimizer

In [ ]:
# pytorch 에서 사용하는 방법
optimizer = torch.optim.Adam()

## 2. 가중치의 초깃값
> 가중치의 초기값이 모두 0이라면 모든 값들이 똑같이 학습될 것이다. 그렇다면 어떤 값으로 어떻게 초기값을 설정해야하는가?

In [1]:
#torch.nn.init.xavier_normal_()
#torch.nn.init.kaiming_normal_()